In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [2]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
len(words)

32033

In [4]:
# build the vocab of chars and mapping to/from ints
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [5]:
# build the dataset
block_size = 3 # context length: how many chars we take to predict the next?
X, Y = [], []
for w in words[:5]:
    print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '---->', itos[ix])
        context = context[1:] + [ix] # crop and append

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ----> e
..e ----> m
.em ----> m
emm ----> a
mma ----> .
olivia
... ----> o
..o ----> l
.ol ----> i
oli ----> v
liv ----> i
ivi ----> a
via ----> .
ava
... ----> a
..a ----> v
.av ----> a
ava ----> .
isabella
... ----> i
..i ----> s
.is ----> a
isa ----> b
sab ----> e
abe ----> l
bel ----> l
ell ----> a
lla ----> .
sophia
... ----> s
..s ----> o
.so ----> p
sop ----> h
oph ----> i
phi ----> a
hia ----> .


In [6]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [7]:
C = torch.randn((27,2))

In [8]:
C[5]

tensor([-1.6694,  0.0249])

In [9]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [10]:
#torch.cat(torch.unbind(emb, 1), 1).shape # this is inefficient vs .view() because it creates a new tensor after concatinating taking up memory
#emb.view(32, 6) == torch.cat(torch.unbind(emb, 1), 1) this comparison returns true showing view functions the same as cat and unbind while reducing memory use. view is just more efficient

In [11]:
W1 = torch.randn((6,100))
b1 = torch.randn(100)

In [12]:
# instead of emb.shape[0] you can put -1 and pytorch will infer the shape
h = torch.tanh(emb.view(emb.shape[0], 6) @ W1 + b1)

In [13]:
h

tensor([[ 0.9835,  0.0508, -0.2276,  ...,  0.9586,  0.4842,  0.1214],
        [ 0.9758,  0.0131, -0.4072,  ...,  0.9284,  0.9687, -0.9270],
        [ 0.9612, -0.7606, -0.1352,  ..., -0.9403,  0.2647,  0.0731],
        ...,
        [-0.1201, -0.9904,  0.0760,  ..., -0.9881, -0.6425, -0.9252],
        [ 0.8985, -0.7628, -0.2841,  ..., -0.9979, -0.5479,  0.9778],
        [-0.3101,  0.7347, -0.1083,  ...,  0.7548, -0.7109,  0.9996]])

In [14]:
h.shape

torch.Size([32, 100])

In [15]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [16]:
logits = h @ W2 + b2

In [17]:
logits.shape

torch.Size([32, 27])

In [18]:
counts = logits.exp()

In [19]:
prob = counts / counts.sum(1, keepdim=True)

In [20]:
prob.shape

torch.Size([32, 27])

In [21]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(14.6009)

In [22]:
# reorganize everything above-------------------------------------------------------------------------------------------------

In [23]:
X.shape, Y.shape # dataset

(torch.Size([32, 3]), torch.Size([32]))

In [24]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27,2), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [25]:
sum(p.nelement() for p in parameters) # num params in total

3481

In [26]:
for p in parameters:
    p.requres_grad = True

In [27]:
for _ in range(10):
    # forward pass
    emb = C[X] # (32, 3, 2)
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # (32, 100)
    logits = h @ W2 + b2 # (32, 27)
    #counts = logits.exp()                                  | these 3 lines are equivalent to F.cross_entropy(logits, Y)
    #prob = counts / counts.sum(1, keepdim=True)            | in F.cross_entropy, pytorch doesn't make all the tensors 
    #loss = -prob[torch.arange(32), Y].log().mean()         | like in the 3 lines here so it is more efficient as extra memory isnt being taken up
    loss = F.cross_entropy(logits, Y) # reasons to do cross_entropy over the 3 lines, forward/backward pass is more efficient, and numerically more well behaved
    print(loss.item())
    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    # update
    for p in parameters:
        p.data += -0.1 * p.grad


3.295837163925171


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn